In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


1. langchain -pipeline  
2.  chromadb-vector store
3.   sentance transformers- embedding model
4. llama-LLM Model (loading)
5.   langchain_community - extra packages
6.   pypdf-for reading pdf






In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292

import the necessary libraries


In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader #to load our directory automatically sees the pdf and reads
from langchain.text_splitter import RecursiveCharacterTextSplitter #divides text into chunks .this splits entire data into chunks
from langchain_community.embeddings import SentenceTransformerEmbeddings #for embeddings
from langchain.vectorstores import Chroma #to store embeddings to vector store
from langchain_community.llms import LlamaCpp #load llm model
from langchain.chains import RetrievalQA, LLMChain #for building end-end application chain means set of instructions

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Import the documents

In [5]:
#don't give the pdf's name.When we give the pdf's location it will automatically find it and read
loader=PyPDFDirectoryLoader("/content/drive/MyDrive/datasets1/")
docs=loader.load()

In [6]:
len(docs) #no of pages docs is actually a list

261

In [8]:
docs[16]

Document(metadata={'source': '/content/drive/MyDrive/datasets1/mh.pdf', 'page': 16}, page_content='I: Mental Health Worldwide \n16 2) Financing \n \nThe global median mental health expenditures per capita are $1.63 \nUSD.  There is a significant difference in median mental health \nexpenditures per capita among income groups, ranging from $0.20 USD in low income countries to $44.84 USD in high income countries. The median percentage of health budget allocated to \nmental health is highest in the EUR (5.0%), 3.75% in EMR, 1.95% \nin WPR, 1.53% in AMR, 0.62% in AFR, and 0.44% in SEAR.  \nRegarding income group, the median percentage of health budget \nallocated to mental health is highest for high income countries \n(5.1%), 2.38% for upper-middle income countries, 1.90% for \nlower-middle countries, and lowest or 0.53% for low income \ncountries.  Sixty-seven percent (67%) of financial resources world-\nwide are directed toward mental hospitals /institutions as opposed \nto community-bas

Chunking

In [9]:
#splitting the text into chunks where each chunk has 300 characters and we give chunk_overlap to maintain the context
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)
chunks = text_splitter.split_documents(docs) #a function to create chunks

In [10]:
len(chunks) #total text in that pdf is split into 1321 chunks

1321

In [12]:
chunks[860]

Document(metadata={'source': '/content/drive/MyDrive/datasets1/mh.pdf', 'page': 177}, page_content='behavior (if any) and action to change that behavior is identified and dis-\ncussed; the HIV test is performed; and the client is asked to return in 2 \nweeks for results.  \nIn the second session, progress made in changing the previously identified')

In [ ]:
#Embeddings creations

In [13]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']="" #use your own token

In [14]:
#generate embeddings
embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
#This is a PubMedBERT-base model fined-tuned using sentence-transformers.
#Further fine-tuning for a medical subdomain will result in even better performance

<ipython-input-14-b5e1a0e1c06e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (htt

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
#Vector Store Creation
vectorstore=Chroma.from_documents(chunks,embeddings) #creates embedding vectors . a vectorstore obj is created
#we pass both so we can do hybrid search

In [16]:
#testing the searching part
query="what do you mean by mental health?"
search_results = vectorstore.similarity_search(query)

In [17]:
search_results

[Document(metadata={'page': 29, 'source': '/content/drive/MyDrive/datasets1/mh.pdf'}, page_content='II. Mental Health Capacity Building \n29 2) What information or skills do you need them to understand and \nWhy  is this information important for them to know? \na) Start with a welcome and introduction (and informal interac-\ntion); \nb) Reiterate why it is important that they have attended;'),
 Document(metadata={'page': 39, 'source': '/content/drive/MyDrive/datasets1/mh.pdf'}, page_content='III. Conditions & Issues: How to Identify Symptoms \n39  \n \n \nHOW TO IDENTIFY PSYCHOLOGICAL SYMPTOMS \n \n \nDescriptions of mental health and illness  \nSomeone with a “healthy mind” has clear thoughts, the ability to \nsolve the problems of daily life, enjoys good relationships with'),
 Document(metadata={'page': 197, 'source': '/content/drive/MyDrive/datasets1/mh.pdf'}, page_content='threatened to talk about behavior they perceive as different.  Feel-\nings of shame, disgrace, or rejection c

In [18]:
#instead of using similarity search we can use retrievel as it has more functionalities
retriever =vectorstore.as_retriever(search_kwargs={'k':5}) #5 means it provides top 5 relevant answers

In [19]:
retriever.get_relevant_documents(query)

<ipython-input-19-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 29, 'source': '/content/drive/MyDrive/datasets1/mh.pdf'}, page_content='II. Mental Health Capacity Building \n29 2) What information or skills do you need them to understand and \nWhy  is this information important for them to know? \na) Start with a welcome and introduction (and informal interac-\ntion); \nb) Reiterate why it is important that they have attended;'),
 Document(metadata={'page': 39, 'source': '/content/drive/MyDrive/datasets1/mh.pdf'}, page_content='III. Conditions & Issues: How to Identify Symptoms \n39  \n \n \nHOW TO IDENTIFY PSYCHOLOGICAL SYMPTOMS \n \n \nDescriptions of mental health and illness  \nSomeone with a “healthy mind” has clear thoughts, the ability to \nsolve the problems of daily life, enjoys good relationships with'),
 Document(metadata={'page': 197, 'source': '/content/drive/MyDrive/datasets1/mh.pdf'}, page_content='threatened to talk about behavior they perceive as different.  Feel-\nings of shame, disgrace, or rejection c

In [20]:
llm=LlamaCpp(
    model_path="/content/drive/MyDrive/datasets1/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048,
    top_p=1
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/datasets1/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

Use LLM and retriver and query,to generate final response

context and query are placeholders </s> is just a separator. user means user info
assistant is a role of LLm


In [21]:
template="""
<|content|>
You are a Mental health support assistant that follows the instructions given and give accurate answers based on the query and context provided
Please give direct and correct answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [22]:
#using chain method
from langchain.schema.runnable import RunnablePassthrough #whatever query we ask must be passed through the stages
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate #for using this template


In [23]:
prompt=ChatPromptTemplate.from_template(template)
#build a prompt using prompt template

In [24]:
rag_chain=(
    {"context": retriever,"query": RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)
#in this chain step-by step is provided in that context placeholder we have retriever and in the query we have runnable... so that the query gets passes to each and every stage
#pipe indicates 1st step completed
#once we get the prompt and the query we pass it to llm and then show the output.
#1st take user query passed through retriever and it gives context

In [25]:
response=rag_chain.invoke(query)
#call the ragchain


llama_print_timings:        load time =    7255.01 ms
llama_print_timings:      sample time =      33.74 ms /    49 runs   (    0.69 ms per token,  1452.24 tokens per second)
llama_print_timings: prompt eval time =   39387.17 ms /    72 tokens (  547.04 ms per token,     1.83 tokens per second)
llama_print_timings:        eval time =   34589.40 ms /    48 runs   (  720.61 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =   74075.59 ms /   120 tokens


In [26]:
response

'Mental health is “our emotional, psychological, and social well-being. It affects how we think, feel, and act, and helps determine how we handle stress, relate to others, and make choices.” (World Health Organization).'

if it should be interactive

In [27]:
import sys
while True:
  user_input =input(f"Input query: ")
  if user_input =='exit':
    print("Exiting...")
    sys.exit()
  if user_input=="":
    continue
  result=rag_chain.invoke(user_input)
  print("Answer: ", result)

Input query: what causes mental health?


Llama.generate: 54 prefix-match hit, remaining 15 prompt tokens to eval

llama_print_timings:        load time =    7255.01 ms
llama_print_timings:      sample time =      73.20 ms /   104 runs   (    0.70 ms per token,  1420.77 tokens per second)
llama_print_timings: prompt eval time =    8876.80 ms /    15 tokens (  591.79 ms per token,     1.69 tokens per second)
llama_print_timings:        eval time =   78913.39 ms /   103 runs   (  766.15 ms per token,     1.31 tokens per second)
llama_print_timings:       total time =   88006.40 ms /   118 tokens


Answer:  There are many factors that can contribute to mental health, including biological, environmental, psychological, and social factors. Biological factors such as genetics, brain chemistry, and hormones can play a role in the development of mental illnesses. Environmental factors such as stress, trauma, and abuse can also contribute to the development of mental illnesses. Psychological factors such as thoughts, beliefs, and behaviors can also impact mental health. Social factors such as relationships, social support, and discrimination can also affect mental health.
Input query: exit
Exiting...


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
